Group 21 Project Report

Introduction

Heart disease is one of the leading causes of death in Canada. It encompasses all kinds of heart conditions such as coronary heart disease, heart defects, and arrythmia’s (irregularity in heartbeat). To better understand heart disease, it is important to look at the key factors that influence an individual’s risk of having this condition. In this project we will be looking at 5 different factors that have a correlation with heart disease. These factors are high blood pressure, high cholesterol, maximum heart rate, age, and chest pain. These will be used as predictors in this project.

So how are each of these  predictors associated with heart disease? The first factor, high blood pressure, places extra exertion on blood vessels, increasing tension on the arteries, which decreases blood flow to major organs and oxygen to the heart. Aside from this, people who have high levels of cholesterol could have fat deposits build up in the arteries, making it  challenging for blood to flow freely through the body. Another thing, most people should be aware of is their maximum heart rate, reaching the ideal heart rate range can indicate if the heart is in good condition. If it is over this, then it could be a sign that there is a heart issue. A common factor in many kinds of diseases is an increase in age, which is often correlated with a higher risk for heart disease. Lastly, incidence of chest pain can be a risk factor for heart disease if evaluated at abnormal levels. The data base classifies the chest pain type by typical chest pain (angina), atypical angina, no chest pain, and asymptomatic.

The question we want to answer is: are we able to predict whether a patient has heart disease based on their health profile? 

The information used to answer this question will be from the heart disease dataset, and used to create an accurate prediction model to classify new data. The heart disease data set encompasses 14 factors, in which 5 of them will be used as predictors and diagnosis of heart disease is what we will be predicting.


References

Aha, D. (2019). Heart Disease Data Set. UCI Machine Learning Repository. https://archive.ics.uci.edu/ml/citation_policy.html 

Lennep, J., Westerveld, H., Erkelens, D., & Van der Wall, E. (2002). Risk factors for coronary heart disease: implications of gender. Cardiovascular Research. 53(3). 538-549.  https://doi.org/10.1016/S0008-6363(01)00388-1

Thayer, J., Yamamoto, S., & Brosschot, J. (2010). The relationship of autonomic imbalance, heart rate variability and cardiovascular disease risk factors. Internal Journal of Cardiology, 141(2). 122-131. https://doi.org/10.1016/j.ijcard.2009.09.543
